# 02 — Analysis & Insights (Instacart)

Explores customer behavior and produces business-facing insights.

**What’s inside:** feature engineering, aggregation, segmentation, and summary tables.

Tip: keep charts + final insights in this notebook for hiring managers.

## Feature Engineering

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [ ]:
# Import datasets
path = r'/Users/spencer/Documents/Career Foundry/Data Immersion/4 Python Fundamentals for Data Analysts/Instacart Basket Analysis'
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge.pkl'))

In [ ]:
# Create subset with 1,000,000 rows
df = ords_prods_merge[:1000000]

In [ ]:
# View data for reference
df.head()

In [ ]:
df.shape

In [ ]:
def price_label(row):

  if row['prices'] <= 5:
    return 'Low-range product'
  elif (row['prices'] > 5) and (row['prices'] <= 15):
    return 'Mid-range product'
  elif row['prices'] > 15:
    return 'High range'
  else: return 'Not enough data'

In [ ]:
# Define new column 'price_range' based on condition. Axis=1 stands for "rows" (0 = apply to columns)
df['price_range'] = df.apply(price_label, axis=1)

In [ ]:
df['price_range'].value_counts(dropna = False)

In [ ]:
df['prices'].max()

In [ ]:
df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [ ]:
df.loc[(df['prices'] <= 15) & (df['prices'] > 5), 'price_range_loc'] = 'Mid-range product'

In [ ]:
df.loc[df['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [ ]:
df['price_range_loc'].value_counts(dropna = False)

In [ ]:
ords_prods_merge.loc[ords_prods_merge['prices'] > 15, 'price_label'] = 'High-range product'

In [ ]:
ords_prods_merge.loc[(ords_prods_merge['prices'] <= 15) & (ords_prods_merge['prices'] > 5), 'price_label'] = 'Mid-range product'

In [ ]:
ords_prods_merge.loc[ords_prods_merge['prices'] <= 5, 'price_label'] = 'Low-range product'

In [ ]:
ords_prods_merge['price_label'].value_counts(dropna = False)

In [ ]:
# View busiest days of week. 0 = Saturday
ords_prods_merge['orders_day_of_week'].value_counts(dropna = False)

In [ ]:
# Create empty list
result = []

# Loop through dataframe
for value in ords_prods_merge["orders_day_of_week"]:
  if value == 0:
    result.append("Busiest day")
  elif value == 4:
    result.append("Least busy")
  else:
    result.append("Regularly busy")

In [ ]:
# Make new column 'busiest day' and combine with result
ords_prods_merge['busiest_day'] = result

In [ ]:
ords_prods_merge['busiest_day'].value_counts(dropna = False)

In [ ]:
# Verify I have “price_label” and “busiest_day” columns.
ords_prods_merge.head()

In [ ]:
# View busiest days of week. 0 = Saturday
ords_prods_merge['orders_day_of_week'].value_counts(dropna = False)

In [ ]:
# Create busiest days column

# Create empty list
result2 = []

# Loop through dataframe, using OR logic
for value in ords_prods_merge["orders_day_of_week"]:
  if value == 0 or value == 1:
    result2.append("Busiest days")
  elif value == 4 or value == 3:
    result2.append("Least busy")
  else:
    result2.append("Regularly busy")

In [ ]:
# Make new column 'busiest days' and combine with result
ords_prods_merge['busiest_days'] = result2

In [ ]:
# View distribution
ords_prods_merge['busiest_days'].value_counts(dropna = False)

In [ ]:
# Expect 32,434,489 rows
ords_prods_merge.shape

In [ ]:
# View spread of data
ords_prods_merge['order_hour_of_day'].value_counts(dropna = False)

In [ ]:
# Define lists of hours, split into thirds (8 hours each group)
most_orders = [10, 11, 14, 15, 13, 12, 16, 9]
average_orders = [17, 8, 18, 19, 20, 7, 21, 22]
fewest_orders = [23, 6, 0, 1, 5, 2, 4, 3]

In [ ]:
# Apply labels using .loc since it is cleaner and faster
ords_prods_merge.loc[ords_prods_merge['order_hour_of_day'].isin(most_orders), 'busiest_period_of_day'] = 'Most orders'
ords_prods_merge.loc[ords_prods_merge['order_hour_of_day'].isin(average_orders), 'busiest_period_of_day'] = 'Average orders'
ords_prods_merge.loc[ords_prods_merge['order_hour_of_day'].isin(fewest_orders), 'busiest_period_of_day'] = 'Fewest orders'

In [ ]:
# Print frequency
ords_prods_merge['busiest_period_of_day'].value_counts()

In [ ]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_merged_updated.pkl'))

## Aggregations

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [ ]:
# Import datasets
path = r'/Users/spencer/Documents/Career Foundry/Data Immersion/4 Python Fundamentals for Data Analysts/Instacart Basket Analysis'
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_updated.pkl'))

In [ ]:
# Create subset of first 1M
df = ords_prods_merge[:1000000]

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.groupby('product_name')

In [ ]:
df.groupby('department_id').agg({'order_number': ['mean']})

In [ ]:
# Same result without agg() function
df.groupby('department_id')['order_number'].mean()

In [ ]:
df.groupby('department_id').agg({'order_number': ['mean', 'min', 'max']})

In [ ]:
# All 3 steps in 1 code
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform("max")

In [ ]:
ords_prods_merge.head(100)

In [ ]:
# Create loyalty flags based on max orders
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [ ]:
# Check values
ords_prods_merge['loyalty_flag'].value_counts()

In [ ]:
# Check head() of only columns of interest using df['column']
ords_prods_merge[['user_id', 'loyalty_flag', 'order_number']].head(60)

In [ ]:
# Task 1 verification
ords_prods_merge.head()

In [ ]:
# 2. Create aggregated mean of “order_number” column grouped by “department_id”
ords_prods_merge.groupby('department_id')['order_number'].mean()

In [ ]:
# 3. Analyze the result
# Compare to subset results:
df.groupby('department_id')['order_number'].mean()

In [ ]:
# 5.The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified.
ords_prods_merge.groupby('loyalty_flag')['prices'].mean()

In [ ]:
ords_prods_merge['prices'].describe()

In [ ]:
ords_prods_merge.loc[ords_prods_merge['prices'] > 100]

In [ ]:
ords_prods_merge.loc[ords_prods_merge['prices'] > 90000]

In [ ]:
# Turn values > 100 into NaNs
ords_prods_merge.loc[ords_prods_merge['prices'] > 100, 'prices'] = np.nan

In [ ]:
# Check for outliers again
ords_prods_merge.loc[ords_prods_merge['prices'] > 50]

In [ ]:
# Complete task 5 again with cleaned data
ords_prods_merge.groupby('loyalty_flag')['prices'].mean()

In [ ]:
# 6. Create a spending flag for each user based on the average price across all their orders
ords_prods_merge['user_avg_item_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform("mean")

In [ ]:
ords_prods_merge.head(100)

In [ ]:
# Create spending flags based on avg order price
ords_prods_merge.loc[ords_prods_merge['user_avg_item_price'] < 10, 'spending_habit'] = 'Low spender'
ords_prods_merge.loc[ords_prods_merge['user_avg_item_price'] >= 10, 'spending_habit'] = 'High spender'

In [ ]:
# Check values
ords_prods_merge['spending_habit'].value_counts()

In [ ]:
# 7. Create an order frequency flag that marks the regularity of a user’s ordering behavior
ords_prods_merge['median_days_between_orders'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform("median")

In [ ]:
ords_prods_merge.head(100)

In [ ]:
# Create user frequency flags based on median days since prior order
ords_prods_merge.loc[ords_prods_merge['median_days_between_orders'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'
ords_prods_merge.loc[(ords_prods_merge['median_days_between_orders'] <= 20) & (ords_prods_merge['median_days_between_orders'] > 10), 'order_frequency_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['median_days_between_orders'] <= 10, 'order_frequency_flag'] = 'Frequent customer'

In [ ]:
ords_prods_merge['order_frequency_flag'].value_counts()

In [ ]:
ords_prods_merge.head()

In [ ]:
# 9. Export your dataframe as a pickle file and store it correctly in your “Prepared Data” folder.
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_aggregated.pkl'))

## Visualizations & Demographics

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [ ]:
# Import datasets
path = r'/Users/spencer/Documents/Career Foundry/Data Immersion/4 Python Fundamentals for Data Analysts/Instacart Basket Analysis'
ords_prods_cust_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_all.pkl'))

In [ ]:
# Create histogram for order_hour_of_day
hours = range(24)

ax = (ords_prods_cust_merge["order_hour_of_day"]
      .value_counts()
      .reindex(hours, fill_value=0)
      .plot(kind="bar"))

ax.set_title("Order Frequency by Hour of Day")
ax.set_xlabel("Hour of Day (0–23)")
ax.set_ylabel("Number of Orders")
plt.tight_layout()
plt.show()

In [ ]:
# Create bar chart for loyalty_flag
bar = ords_prods_cust_merge['loyalty_flag'].value_counts().plot.bar()

In [ ]:
# Create the seed (ensures reproducibility)
np.random.seed(4)

In [ ]:
# Create a list holding True/False values to the test np.random.rand() <= 0.7
dev = np.random.rand(len(ords_prods_cust_merge)) <= 0.7

In [ ]:
# Store 70% of the sample in the dataframe 'big'
big = ords_prods_cust_merge[dev]

In [ ]:
# Store 30% of the sample in the dataframe 'small'
small = ords_prods_cust_merge[~dev]

In [ ]:
# Verify row counts
len(ords_prods_cust_merge)

In [ ]:
len(big) + len(small)

In [ ]:
# Create the line chart using the small sample
line = sns.lineplot(data = small, x = 'order_hour_of_day', y = 'prices')

In [ ]:
# Create line chart for Age vs. Number of Dependents
line_2 = sns.lineplot(data = small, x = 'age', y = 'number_of_dependents')

In [ ]:
# Create scatterplot for Age vs. Income
scatter = sns.scatterplot(x = 'age', y = 'income', data = ords_prods_cust_merge)

In [ ]:
hist.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'hist_order_hour_of_day.png'))

In [ ]:
bar.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'bar_loyalty_flag.png'))

In [ ]:
line.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'line_prices_hour_of_day.png'))

In [ ]:
line_2.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'line_age_dependents.png'))

In [ ]:
scatter.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'scatter_age_income.png'))

## Regional Segmentation

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [ ]:
# Import datasets
path = r'/Users/spencer/Documents/Career Foundry/Data Immersion/4 Python Fundamentals for Data Analysts/Instacart Basket Analysis'
df = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_all.pkl'))
df_dept = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'departments.csv'), index_col = False)

In [ ]:
df.columns

In [ ]:
# Drop PII columns
df = df.drop(columns=['first_name', 'surname'])

In [ ]:
# Check to ensure they are gone
df.columns

In [ ]:
# View States
df['state'].value_counts()

In [ ]:
region_northeast = ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']

In [ ]:
region_midwest = ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri']

In [ ]:
region_south = ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']

In [ ]:
region_west = ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico', 'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii']

In [ ]:
df.loc[df['state'].isin(region_northeast), 'region'] = 'Northeast'

In [ ]:
df.loc[df['state'].isin(region_midwest), 'region'] = 'Midwest'

In [ ]:
df.loc[df['state'].isin(region_south), 'region'] = 'South'

In [ ]:
df.loc[df['state'].isin(region_west), 'region'] = 'West'

In [ ]:
# Check the output
df['region'].value_counts(dropna=False)

In [ ]:
# Create a crosstab
crosstab = pd.crosstab(df['region'], df['spending_habit'], dropna = False)

In [ ]:
crosstab

In [ ]:
# Create a bar chart
bar_chart = crosstab.plot.bar(color=['lightblue', 'darkblue'])

In [ ]:
# Export chart
bar_chart.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'bar_regional_spending_habits.png'))

In [ ]:
# Create the activity flag
df.loc[df['max_order'] < 5, 'activity_flag'] = 'Low activity'
df.loc[df['max_order'] >= 5, 'activity_flag'] = 'High activity'

In [ ]:
# Check the count of each flag
df['activity_flag'].value_counts(dropna = False)

In [ ]:
# Create a subset of ONLY high activity customers
df_active = df[df['activity_flag'] == 'High activity']

In [ ]:
# Check the shape to verify rows were dropped
df_active.shape

In [ ]:
# Export the active customers data
df_active.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_active.pkl'))

In [ ]:
# Shift dataframe to only high activity customers
df = df_active

In [ ]:
# Create age_group column
df.loc[df['age'] <= 35, 'age_group'] = 'Young Adult'
df.loc[(df['age'] > 35) & (df['age'] < 65), 'age_group'] = 'Middle Aged'
df.loc[df['age'] >= 65, 'age_group'] = 'Senior'

In [ ]:
# Check the distribution
df['age_group'].value_counts(dropna=False)

In [ ]:
# Create income_group column
df.loc[df['income'] < 50000, 'income_group'] = 'Low Income'
df.loc[(df['income'] >= 50000) & (df['income'] <= 120000), 'income_group'] = 'Middle Income'
df.loc[df['income'] > 120000, 'income_group'] = 'High Income'

In [ ]:
# Check the distribution
df['income_group'].value_counts(dropna=False)

In [ ]:
# Create dependent_flag
df.loc[df['number_of_dependents'] > 0, 'dependent_flag'] = 'Parent'
df.loc[df['number_of_dependents'] == 0, 'dependent_flag'] = 'Non-parent'

In [ ]:
# Check the distribution
df['dependent_flag'].value_counts(dropna=False)

In [ ]:
# Young Parent
df.loc[(df['age_group'] == 'Young Adult') & (df['dependent_flag'] == 'Parent'), 'customer_profile'] = 'Young Parent'

In [ ]:
# Single Adult
df.loc[(df['age_group'] == 'Young Adult') & (df['dependent_flag'] == 'Non-parent'), 'customer_profile'] = 'Single Adult'

In [ ]:
# Middle Aged Parent
df.loc[(df['age_group'] == 'Middle Aged') & (df['dependent_flag'] == 'Parent'), 'customer_profile'] = 'Middle Aged Parent'

In [ ]:
# Single Senior
df.loc[(df['age_group'] == 'Senior') & (df['dependent_flag'] == 'Non-parent'), 'customer_profile'] = 'Single Senior'

In [ ]:
# Senior Parent
df.loc[(df['age_group'] == 'Senior') & (df['dependent_flag'] == 'Parent'), 'customer_profile'] = 'Senior Parent'

In [ ]:
# Single Middle Aged
df.loc[(df['age_group'] == 'Middle Aged') & (df['dependent_flag'] == 'Non-parent'), 'customer_profile'] = 'Single Middle Aged'

In [ ]:
# Check the distribution
df['customer_profile'].value_counts(dropna=False)

In [ ]:
# Create customer profile summary
table_profile_summary = df.groupby('customer_profile').agg({'max_order': ['mean', 'min', 'max'], 'prices': ['mean', 'min', 'max'], 'days_since_prior_order': ['mean']}).round(2)

In [ ]:
table_profile_summary

In [ ]:
# Create a flag for users who have ordered from the baby department (ID 18)
df['baby_department'] = [1 if x == 18 else 0 for x in df['department_id']]

In [ ]:
# Assign flag if user has ever bought from dept 18
df['has_baby_status'] = df.groupby(['user_id'])['baby_department'].transform('max')

In [ ]:
# Label the profile for clarity
df.loc[df['has_baby_status'] == 1, 'baby_status'] = 'Baby Household'
df.loc[df['has_baby_status'] == 0, 'baby_status'] = 'Non-baby Household'

In [ ]:
# Drop the temp column
df = df.drop(columns=['baby_department'])

In [ ]:
# Check the distribution
df['baby_status'].value_counts(dropna=False)

In [ ]:
# Create a bar chart for Customer Profile
bar_profile = df['customer_profile'].value_counts().plot.bar(title='Distribution of Customer Profiles')

In [ ]:
# Save the visualization
bar_profile.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'bar_customer_profile.png'))

In [ ]:
# Group by Profile AND Income to see the spending differences
df.groupby(['customer_profile', 'income_group']).agg({'prices': ['mean', 'min', 'max'], 'days_since_prior_order': ['mean', 'min', 'max']})

In [ ]:
# Aggregation for baby status
df.groupby(['baby_status', 'income_group']).agg({'prices': ['mean', 'max', 'min'], 'days_since_prior_order': ['mean', 'max', 'min']})

In [ ]:
# Create a crosstab to compare Customer Profiles and Regions
crosstab_region = pd.crosstab(df['customer_profile'], df['region'], dropna=False)

In [ ]:
# Check the table
crosstab_region

In [ ]:
# Optional: enforce a consistent region order (adjust if yours differ)
region_order = ["Midwest", "Northeast", "South", "West"]
cols = [c for c in region_order if c in crosstab_region.columns] or list(crosstab_region.columns)

# Sort profiles by total orders across regions (descending)
ct = (crosstab_region[cols]
      .assign(_total=crosstab_region[cols].sum(axis=1))
      .sort_values("_total", ascending=False)
      .drop(columns="_total"))


# ax = ct.plot(kind="bar", color=['#DEEBF7', '#9ECAE1', '#4292C6', '#084594'])

ax.set_title("Customer Profile by Region")
ax.set_xlabel("Customer Profile")
ax.set_ylabel("Number of Orders")
ax.legend(title="Region", frameon=False)

plt.xticks(rotation=25, ha="right")
ax.yaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{x/1_000_000:.1f}M"))

plt.tight_layout()
plt.show()

In [ ]:
# Export the chart
bar_region_profile.figure.savefig(os.path.join(path, '04 Analysis', 'Visualizations', 'bar_profile_region.png'))

In [ ]:
# Transpose the departments dataframe
df_dept_t = df_dept.T
df_dept_t.reset_index()

In [ ]:
# Create a new header
new_header = df_dept_t.iloc[0] 
df_dept_t = df_dept_t[1:] 
df_dept_t.columns = new_header

In [ ]:
# Check the output to ensure it looks right
df_dept_t

In [ ]:
# Create dictionary
data_dict = df_dept_t.to_dict('index')

In [ ]:
# Flatten result (instead of {1: {'department': 'frozen'}}, we want {1: 'frozen'})
department_dict = {int(key): value['department'] for key, value in data_dict.items()}

In [ ]:
# Map the dictionary to main dataframe
df['department'] = df['department_id'].map(department_dict)

In [ ]:
# Check the result
df[['department_id', 'department']].head()

In [ ]:
# Verify the count
df['department'].value_counts(dropna=False)

In [ ]:
# Fill NaNs with the string 'missing'
df['department'] = df['department'].fillna('missing')

In [ ]:
# Verify the count
df['department'].value_counts(dropna=False)

In [ ]:
# Create a crosstab to compare Customer Profiles and Departments
crosstab_dept = pd.crosstab(df['department'], df['customer_profile'], dropna=False)

In [ ]:
crosstab_dept

In [ ]:
# Create a stacked bar chart
bar_dept_profile = crosstab_dept.plot.bar(stacked=True)
plt.title('Department Orders by Customer Profile')
plt.ylabel('Frequency')

In [ ]:
# Export stacked bar chart
bar_dept_profile.figure.savefig(os.path.join(path, '04 Analysis', 'Visualizations', 'bar_dept_profile.png'))

In [ ]:
df.shape

In [ ]:
# Create loyalty behavior summary table
table_loyalty_summary = df.groupby('loyalty_flag').agg({
    'max_order': ['mean', 'min', 'max'],
    'prices': ['mean', 'min', 'max'],
    'days_since_prior_order': ['mean']
}).round(2)

In [ ]:
table_loyalty_summary

In [ ]:
# Make price range vs price_label table
price_counts = df['price_label'].value_counts(dropna=False)

In [ ]:
price_pct = (df['price_label'].value_counts(normalize=True, dropna=False) * 100).round(1)

In [ ]:
price_label_mix = pd.concat([price_counts, price_pct], axis=1)

In [ ]:
price_label_mix.columns = ['order_lines', 'percent']

In [ ]:
price_label_mix

In [ ]:
# Make revenue by hour (real order totals + AOV)

# Build order-level totals (sum of prices per order_id)
df_orders = (
    df.groupby("order_id", as_index=False)
      .agg(
          order_hour_of_day=("order_hour_of_day", "first"),
          order_total=("prices", "sum")
      )
)

In [ ]:
# Summarize by hour
hourly_rev = (
    df_orders.groupby("order_hour_of_day", as_index=False)
             .agg(
                 total_revenue=("order_total", "sum"),
                 avg_order_value=("order_total", "mean"),
                 median_order_value=("order_total", "median"),
                 n_orders=("order_total", "size")
             )
             .sort_values("order_hour_of_day")
)

In [ ]:
hourly_rev

In [ ]:
hourly_rev.plot(x="order_hour_of_day", y="total_revenue", kind="bar", figsize=(10,4), legend=False, title="Total revenue by hour")

In [ ]:
hourly_rev.plot(x="order_hour_of_day", y="avg_order_value", kind="line", figsize=(10,4), legend=False, title="Average order value (AOV) by hour")

In [ ]:
# Export the final data set with the fixed 'department' column
df.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_final_profiles.pkl'))